<a href="https://colab.research.google.com/github/danizc/Facial-Recognition/blob/main/Reconhecimento_Facial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRABALHO DE ROBÓTICA - Projeto**

Alterar tipo de ambiente de execução para GPU

In [ ]:
!pip install face_recognition

In [14]:
import face_recognition
import os
import cv2
from google.colab.patches import cv2_imshow
from pathlib import Path

In [15]:
KNOWN_FACES_DIR = Path('known_faces')
UNKNOWN_FACES_DIR = Path('unknown_faces')
KNOWN_FACES_DIR.mkdir(exist_ok=True)
UNKNOWN_FACES_DIR.mkdir(exist_ok=True)

#Reconhecimento facial para imagens individuais

In [ ]:
TOLERANCE = 0.5 #nível de rigorosidade 
FRAME_THICKNESS = 3 #tamanho da linha do retângulo
FONT_THICKNESS = 2 #tamanho da linha da fonte
MODEL = 'cnn' # (convolutional neural network deep leaning)


def name_to_color(name):
    
    color = [(ord(c.lower())-97)*8 for c in name[:3]] #colorir o retângulo demarcando o rosto
    return color

print('Loading known faces...') #leitura dos arquivos/listas 
known_faces = [] #lista de rostos que conhecemos
known_names = [] #lista de rostos para ser identificados

for name in os.listdir(KNOWN_FACES_DIR): #iterar known_faces
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'): #carregar files

      if not os.path.isfile(f'{KNOWN_FACES_DIR}/{name}/{filename}'): #apenas verifica/separa o que não é um file
        continue

      image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
      encoding = face_recognition.face_encodings(image)[0] #identifica o primeiro rosto e retorna lista de rostos encontrados
      known_faces.append(encoding) #adicionam o nome aos rostos encontrados se identificado
      known_names.append(name)

print('Processing unknown faces...')

for filename in os.listdir(UNKNOWN_FACES_DIR): #iterar lista com faces à serem reconhecidas

  if not os.path.isfile(f'{UNKNOWN_FACES_DIR}/{filename}'): #apenas verifica/separa o que não é um file
    continue

  print(f'Filename {filename}', end='')

  image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}')
  locations = face_recognition.face_locations(image, model=MODEL) #localizar rostos nas imagens
  encodings = face_recognition.face_encodings(image, locations) #identificar rostos
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #desenhar retângulo ao redor do rosto

  print(f', found {len(encodings)} face(s)')

  for face_encoding, face_location in zip(encodings, locations): #assumir que pode ter mais rostos na imagem

    results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE) #comparar imagens e avalia-las
    match = None # É feita uma checagem do rosto que foi encontrado, e atribui a este rosto conhecido uma tolerância

    if True in results: # Se pelo menos um dos resultados é verdadeiro, relaciona o nome correto a imagem através da tolerância
        match = known_names[results.index(True)]
        print(f' - {match} from {results}')

        # Constrói o retângulo, sendo que depende de uma ordem pré-estabelecida dos lados: top, right, bottom, left
        top_left = (face_location[3], face_location[0])
        bottom_right = (face_location[1], face_location[2])
        color = name_to_color(match) # Atribui uma cor através de uma função
        cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS) # Pinta os frames do retângulo
        top_left = (face_location[3], face_location[2]) # Faz um retângulo menor para que o nome da imagem fique em destaque dentro dele
        bottom_right = (face_location[1], face_location[2] + 22) # Puxa os dois cantos inferiores 50 pixels para baixo de maneira que o retângulo fique preenchido, dando mais destaque ao nome
        cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED) # Pinta os frames do retângulo de forma a preenchê-lo
        cv2.putText(image, match, (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), FONT_THICKNESS) # Atribui um nome à imagem
        cv2_imshow(image) # Mostra apenas as imagens que foram identificadas
        